In [ ]:
from bs4 import BeautifulSoup
import requests
# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd
import os
import re
import numpy as np
import time
import unidecode
from tqdm import tqdm
import csv

# Random integer for more realistic timing for clicks, buttons and searches during scraping
import random
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait
import math
from datetime import date
import json

In [2]:
import selenium

# Check version I am running
selenium.__version__

'4.31.1'

In [3]:
import undetected_chromedriver as uc


In [4]:
from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.action_chains import ActionChains

In [5]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [6]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/browser-headers',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
header_browser_list = response.json()
print(header_browser_list['result'][0])

{'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': '"Windows"', 'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'sec-fetch-site': 'same-site', 'sec-fetch-mode': 'navigate', 'sec-fetch-user': '?1', 'sec-fetch-dest': 'document', 'accept-encoding': 'gzip, deflate, br, zstd', 'accept-language': 'en-US'}


In [7]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/user-agents',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
user_agent_list = response.json()
print(user_agent_list['result'][0])

Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Mobile Safari/537.36


In [8]:
random_index_user_agent = randint(0, len(user_agent_list)-1)
random_index_header_browser = randint(0, len(header_browser_list)-1)
user_agent_random = user_agent_list['result'][random_index_user_agent]
header_browser_random = header_browser_list['result'][random_index_header_browser]

In [9]:
def create_option():
    random_index_user_agent = randint(0, len(user_agent_list)-1)
    random_index_header_browser = randint(0, len(header_browser_list)-1)
    user_agent_random = user_agent_list['result'][random_index_user_agent]
    header_browser_random = header_browser_list['result'][random_index_header_browser]
    # Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
    option = uc.ChromeOptions()

    # Going undercover:
    #option.add_argument("--incognito")


    # # Consider this if the application works and you know how it works for speed ups and rendering!

    # option.add_argument('--headless=chrome')
    user_agent = user_agent_random
    option.add_argument(f"user-agent={user_agent}")

    # Thêm header vào Options
    headers = header_browser_random

    for key, value in headers.items():
        option.add_argument(f"--header={key}:{value}")
    
    return option

In [10]:
driver = uc.Chrome(service=ChromeService(ChromeDriverManager().install()),options=create_option())
# driver.implicitly_wait(10)  # Set the implicit wait time to 10 seconds
# driver.set_page_load_timeout(30)
driver.get("https://www.tripadvisor.com")
driver.maximize_window()


In [23]:
# Tìm và nhấn vào tìm kiếm nhà hàng với XPath
hotel_ele = driver.find_element(By.XPATH, "//a[@href='/Hotels']")
hotel_ele.click()

# Nhập thành phố
country = 'Cambodia'
place = 'Battambang'
time.sleep(random.uniform(0.8, 2))  # Thời gian trễ ngẫu nhiên

# # Tìm thanh search
# search_bar_location = driver.find_element(By.XPATH, '//div[@class="F1 f u Q2"]')

# # Thêm thời gian trễ ngẫu nhiên giữa các hành động
# time.sleep(random.uniform(0.8, 2))
# search_bar_location.clear()

# time.sleep(random.uniform(0.8, 2))
# search_bar_location.send_keys(f"{place}")

# # Di chuyển chuột đến thanh tìm kiếm để giả lập người dùng thật
# actions = ActionChains(driver)
# actions.move_to_element(search_bar_location).perform()
# time.sleep(random.uniform(0.8, 2))

# # Tìm liên kết "Hotels" và bấm vào
# restaurant_link = driver.find_element(By.XPATH, "//div[@class='yOZMu u j']//a[1]")
# time.sleep(random.uniform(2, 4))

# # Bấm vào link "Hotels"
# restaurant_link.click()
# time.sleep(random.uniform(2, 4))


# Tìm ô input thực sự bên trong search box
search_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//input[contains(@placeholder, "Where to?") or contains(@aria-label, "Search")]'))
)

# Gõ địa điểm vào ô input
search_input.send_keys(Keys.CONTROL, 'a')  # Chọn hết text
search_input.send_keys(Keys.BACKSPACE)     # Xoá
time.sleep(random.uniform(0.8, 2))
search_input.send_keys(place)


# Tìm liên kết "Hotels" và bấm vào
restaurant_link = driver.find_element(By.XPATH, "//div[@class='yOZMu u j']//a[1]")
time.sleep(random.uniform(2, 4))

# Bấm vào link "Hotels"
restaurant_link.click()
time.sleep(random.uniform(2, 4))


In [24]:
# Hàm lấy thông tin URL trong trang
def getURL():
    page_source = BeautifulSoup(driver.page_source, 'html.parser') 
    restaurants = page_source.find_all('a', class_='BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS')
    all_restaurant_url = []

    
    for restaurant in restaurants:
        restaurant_link = restaurant.get('href')
        # Kiểm tra nếu không có dấu '#' trong URL
        if restaurant_link and '#' not in restaurant_link:
            restaurant_url = 'https://www.tripadvisor.com' + restaurant_link
            if restaurant_url not in all_restaurant_url:
                all_restaurant_url.append(restaurant_url)
                
    return all_restaurant_url

getURL()


['https://www.tripadvisor.com/Hotel_Review-g303666-d1848860-Reviews-Bambu_Battambang_Hotel-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d1766117-Reviews-The_Sanctuary_Villa-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d24981430-Reviews-Muni_Residence_Spa-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d17506561-Reviews-Oh_Battambang_Boutique_Hotel-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d6126819-Reviews-Classy_Hotel_Spa-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d32711358-Reviews-Cambana_La_Riviere_Hotel-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d25346434-Reviews-Lotus_Blanc_Homestay-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d3569048-Reviews-Delux_Villa-Battambang_Battamban

In [25]:
# Hàm lấy URL của tất cả các trang
URLs_all_page = []

def getUrlOnPages():
    input_page = 1000

    for page in range(input_page):
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  
            last_height = new_height

        URLs_one_page = getURL()  
        # URLs_all_page.extend(URLs_one_page)  # Thêm URL của trang hiện tại vào danh sách tổng
        
        
        # Kiểm tra và chỉ thêm những URL chưa có trong danh sách
        for url in URLs_one_page:
            if 'Hotel_Review' in url and url not in URLs_all_page:
                URLs_all_page.append(url)
                
        try:
            next_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
            next_page_url = next_button.get_attribute('href')  
            driver.get(next_page_url)  
            time.sleep(random.uniform(2, 4))
        except:
            print(f"Không tìm thấy nút 'next' ở trang {page + 1}. Dừng scrape.")
            break  

    return URLs_all_page

# Gọi hàm
getUrlOnPages()


Không tìm thấy nút 'next' ở trang 6. Dừng scrape.


['https://www.tripadvisor.com/Hotel_Review-g303666-d1848860-Reviews-Bambu_Battambang_Hotel-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d1766117-Reviews-The_Sanctuary_Villa-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d24981430-Reviews-Muni_Residence_Spa-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d17506561-Reviews-Oh_Battambang_Boutique_Hotel-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d6126819-Reviews-Classy_Hotel_Spa-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d32711358-Reviews-Cambana_La_Riviere_Hotel-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d25346434-Reviews-Lotus_Blanc_Homestay-Battambang_Battambang_Province.html',
 'https://www.tripadvisor.com/Hotel_Review-g303666-d3569048-Reviews-Delux_Villa-Battambang_Battamban

In [26]:
len(URLs_all_page)

145

In [27]:
# Kiểm tra xem file đã có dữ liệu chưa (để tránh ghi lại tiêu đề)
file_exists = os.path.isfile("Hotel_Data\Hotel_Tripadvisor_Data_SEA.csv")

with open("Hotel_Data\Hotel_Tripadvisor_Data_SEA.csv", 'a', newline='', encoding='utf-8-sig') as file_output:
    headers = [
        "Hotel ID", "Country", "Tourist Attraction", "Name", "Address", "Price", "Phone Number", "Website", "Ranking", 
        "Description", "Room Features", "Room Types", "Hotel Class", "Languages Spoken", 
        "Hotel Style", "Overall Rating Value", "Overall Rating Text", "Overall Review Count", 
        "Excellent Reviews", "Very Good Reviews", "Average Reviews", "Poor Reviews", "Terrible Reviews", 
        "Walker Score", "Airport Names", "Airport Distances", 
        "Near Hotels", "Near Restaurants", "Near Attractions",
        "Hotels Group", "Restaurants Group", "Attractions Group"
    ]
    writer = csv.DictWriter(file_output, delimiter=",", lineterminator="\n", fieldnames=headers)

    # Ghi tiêu đề chỉ khi file trống (chưa có dữ liệu)
    if not file_exists:
        writer.writeheader()
        
    # Khởi tạo biến đếm cho Hotel ID
    hotel_id_counter = 151343

    # Thiết lập thanh tiến độ
    with tqdm(total=len(URLs_all_page), desc="Crawling hotel data", dynamic_ncols=True, unit="hotel", leave=False) as pbar:
        for hotel_url in URLs_all_page:
            driver.get(hotel_url)
            time.sleep(4)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
        
            # Lấy thông tin tên 
            name = re.sub(r'\s+', ' ', driver.find_element(By.CSS_SELECTOR, '#HEADING').text).strip()

            
            address_elem = driver.find_element(
                By.XPATH,
                '//button//span[contains(text(), ",")]'
            )

            address = address_elem.text.strip()


            price_elem = soup.find("div", attrs={"data-automation": "metaPrice"})
            price = price_elem.get_text(strip=True) if price_elem else np.nan


            try:
                phone_number = driver.find_element(By.CSS_SELECTOR, 'div[class="kpIff f u"] div').text.strip()
                if not phone_number:  
                    phone_number = np.nan
            except:
                phone_number = np.nan 

            try:
                website_element = driver.find_element(By.CSS_SELECTOR, 'div.eeVey.S4.H3.f.u.UKvGa a')
                website = website_element.get_attribute('href') 
            except NoSuchElementException:
                website = np.nan  
                
            try:
                ranking = driver.find_element(By.CSS_SELECTOR, 'div[class="d"] div[class="biGQs _P pZUbB KxBGd"]').text
                if not ranking:
                    ranking = np.nan
            except:
                ranking = np.nan
                
            description_element = soup.find("div", class_="fIrGe _T")
            if description_element:
                description = re.sub(r'\s+', ' ', description_element.get_text()).strip()
            else:
                description = np.nan
                
            # Di chuyển chuột đến giữa trang
            ActionChains(driver).move_to_element(driver.find_element(By.TAG_NAME, 'body')).perform()

            # Cuộn xuống cuối trang
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Lấy thông tin về phòng
            room_features = []
            room_types = []

            all_containers = soup.find_all("div", class_="Jevoh f K")

            # Trích xuất Room Features:
            if len(all_containers) >= 1:
                room_features = [div.get_text(strip=True) for div in all_containers[0].find_all("div", attrs={"data-test-target": "amenity_text"})]
            else:
                room_features = np.nan

            # Trích xuất Room Types:
            if len(all_containers) >= 2:
                room_types = [div.get_text(strip=True) for div in all_containers[1].find_all("div", attrs={"data-test-target": "amenity_text"})]
            else:
                room_types = np.nan


            # Khởi tạo các biến mặc định
            hotel_class = np.nan
            hotel_style = np.nan
            language_spoken = np.nan

            # 1. Lấy HOTEL CLASS:
            for container in soup.select("div.uqMDf.z.RHFuI.YGfmd.YQkjl"):
                if container.find("span", string=re.compile("HOTEL CLASS", re.IGNORECASE)):
                    rating_container = container.select_one("div.CMiVw._R.MC.S4._a.H")
                    if rating_container:
                        title_elem = rating_container.find("title")
                        if title_elem:
                            hotel_class = title_elem.get_text(strip=True)
                    break

            # 2. Lấy HOTEL STYLE:
            # Tìm container chứa nhãn "HOTEL STYLE"
            for container in soup.select("div.uqMDf.z.RHFuI.YGfmd.YQkjl"):
                style_label = container.find("div", string=re.compile("HOTEL STYLE", re.IGNORECASE))
                if style_label:
                    # Lấy các phần tử nằm sau nhãn HOTEL STYLE trong cùng container
                    style_values = style_label.find_next_siblings("div", class_=re.compile("CMiVw"))
                    if style_values:
                        hotel_style = ", ".join(elem.get_text(strip=True) for elem in style_values)
                    break

            # 3. Lấy LANGUAGES SPOKEN:
            for container in soup.select("div.uqMDf.z.RHFuI.YGfmd.YQkjl"):
                lang_label = container.find("div", string=re.compile("LANGUAGES SPOKEN", re.IGNORECASE))
                if lang_label:
                    lang_elem = lang_label.find_next_sibling("div", class_=re.compile("CMiVw"))
                    if lang_elem:
                        language_spoken = lang_elem.get_text(strip=True)
                    break


            # 1. Overall Rating Value:
            overall_rating_value_elem = soup.select_one("div.qymjm > div.biGQs._P.fiohW.uGzyt")
            overall_rating_value = overall_rating_value_elem.get_text(strip=True) if overall_rating_value_elem else np.nan

            # 2. Overall Rating Text:
            overall_rating_text_elem = soup.select_one("div.qymjm > div.biGQs._P.fiohW.fOtGX")
            overall_rating_text = overall_rating_text_elem.get_text(strip=True) if overall_rating_text_elem else np.nan

            # 3. Overall Review Count:
            # Chúng ta lấy phần chứa số review ở bên trong khối JbuXZ
            overall_review_count_elem = soup.select_one("div.JbuXZ div.biGQs._P.pZUbB.KxBGd div.biGQs._P.pZUbB.KxBGd")
            if overall_review_count_elem:
                overall_review_count = overall_review_count_elem.get_text(strip=True)
                # Loại bỏ cặp dấu ngoặc nếu có
                overall_review_count = overall_review_count.strip("()")
            else:
                overall_review_count = np.nan



            excellent_reviews = np.nan
            very_good_reviews = np.nan
            average_reviews = np.nan
            poor_reviews = np.nan
            terrible_reviews = np.nan

            # # Duyệt qua tất cả các <li> trong <ul class="teJcc S4 Cj">
            # for li in soup.select("ul.teJcc.S4.Cj li"):
            #     label_elem = li.find("label")
            #     count_elem = li.find("span", class_="QErCz")
            #     if label_elem and count_elem:
            #         label = label_elem.get_text(strip=True)
            #         count = count_elem.get_text(strip=True)
            #         # Gán giá trị dựa theo nhãn
            #         if label == "Excellent":
            #             excellent_reviews = count
            #         elif label == "Good":
            #             very_good_reviews = count
            #         elif label == "Average":
            #             average_reviews = count
            #         elif label == "Poor":
            #             poor_reviews = count
            #         elif label == "Terrible":
            #             terrible_reviews = count


            for item in soup.select("div.lMovV div.jxnKb"):
                label_elem = item.select_one("div.o div.Ygqck.o.W.q")
                count_elem = item.select_one("div.q div.biGQs._P.fiohW.biKBZ.osNWb")
                if label_elem and count_elem:
                    label = label_elem.get_text(strip=True)
                    count = count_elem.get_text(strip=True)
                    if label == "Excellent":
                        excellent_reviews = count
                    elif label == "Good":
                        very_good_reviews = count
                    elif label == "Average":
                        average_reviews = count
                    elif label == "Poor":
                        poor_reviews = count
                    elif label == "Terrible":
                        terrible_reviews = count


                        
            # walker_score = soup.select_one("div.KXkdH.f.Q2 div.lRSQh.f.j div.biGQs._P.fiohW.ngXxk")
            # walker_score = walker_score.get_text(strip=True) if walker_score else np.nan

            # # Lấy thông tin về sân bay
            # airport_divs = soup.select("div.f.e div.MGDaH.f.u.k.q.Pj.PN.Pw.PA")
            # airport_names = []
            # airport_distances = []
            # for div in airport_divs:
            #     name_elem = div.select_one("span.biGQs._P.fiohW.fOtGX")
            #     distance_elem = div.select_one("span.biGQs._P.pZUbB.KxBGd")
            #     name_val = name_elem.get_text(strip=True) if name_elem else np.nan
            #     distance_val = distance_elem.get_text(strip=True) if distance_elem else np.nan
            #     airport_names.append(name_val)
            #     airport_distances.append(distance_val)
            # if not airport_names:
            #     airport_names = np.nan
            # if not airport_distances:
            #     airport_distances = np.nan

            # # Lấy danh sách cho các nhóm khách sạn, nhà hàng, địa điểm du lịch
            # group_results = {
            #     "Hotels": [],
            #     "Restaurants": [],
            #     "Attractions": []
            # }
            # hotel_count = np.nan
            # restaurant_count = np.nan
            # attraction_count = np.nan

            # for dt in soup.find_all("dt"):
            #     button = dt.find("button")
            #     if not button:
            #         continue
            #     header_text = button.get_text(separator=" ", strip=True)
            #     category = None
            #     if "Hotels" in header_text:
            #         category = "Hotels"
            #     elif "Restaurants" in header_text:
            #         category = "Restaurants"
            #     elif "Attractions" in header_text:
            #         category = "Attractions"
            #     else:
            #         continue
            #     count_match = re.search(r"\((\d+)\)", header_text)
            #     if count_match:
            #         count_val = int(count_match.group(1))
            #         if category == "Hotels":
            #             hotel_count = count_val
            #         elif category == "Restaurants":
            #             restaurant_count = count_val
            #         elif category == "Attractions":
            #             attraction_count = count_val
            #     dd = dt.find_next_sibling("dd")
            #     if not dd:
            #         continue
            #     for a in dd.find_all("a", class_="BMQDV _F Gv wSSLS SwZTJ"):
            #         detail_url = a.get("href", np.nan)
            #         name_elem = a.find("div", class_=lambda c: c and "alXOW" in c)
            #         group_name = name_elem.get_text(strip=True) if name_elem else np.nan
            #         rating_elem = a.find("div", class_="biGQs _P pZUbB hmDzD")
            #         group_rating = rating_elem.get_text(strip=True) if rating_elem else np.nan
            #         review_count = np.nan
            #         review_parent = a.find("div", class_="nKWJn u qMyjI")
            #         if review_parent:
            #             review_elem = review_parent.find("div", class_="biGQs _P pZUbB hmDzD")
            #             if review_elem:
            #                 review_count = review_elem.get_text(strip=True).strip("()")
            #         walk_time = np.nan
            #         walk_parent = a.find("div", class_="biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU hmDzD")
            #         if walk_parent:
            #             span_walk = walk_parent.find("span", class_="NnILp")
            #             if span_walk:
            #                 walk_time = span_walk.get_text(strip=True)
            #         group_results[category].append({
            #             "detail_url": detail_url,
            #             "name": group_name,
            #             "rating": group_rating,
            #             "review_count": review_count,
            #             "walk_time": walk_time
            #         })

            # --- Lấy Walker Score (Grade) ---
            walker_score = np.nan
            # Phần Walker Score trong HTML mới nằm trong div.msCKX (nếu có) hoặc trong một container có chữ "Grade:"
            msckx_elem = soup.find("div", class_="msCKX")
            if msckx_elem:
                text = msckx_elem.get_text(strip=True)
                match = re.search(r"Grade:\s*(\d+)", text, re.IGNORECASE)
                if match:
                    walker_score = match.group(1)
            else:
                # Nếu không có, có thể thử tìm trực tiếp theo chuỗi "Grade:" trên toàn trang
                grade_text = soup.find(string=re.compile(r"Grade:\s*\d+", re.IGNORECASE))
                if grade_text:
                    match = re.search(r"Grade:\s*(\d+)", grade_text, re.IGNORECASE)
                    if match:
                        walker_score = match.group(1)

            # --- Lấy thông tin sân bay ---
            # Giả sử toàn bộ thông tin sân bay nằm trong container có class "YDago"
            airport_container = soup.find("div", class_="YDago")
            if airport_container:
                # Lấy danh sách tên sân bay
                airport_name_elems = airport_container.find_all("div", class_="MAfzX")
                if airport_name_elems:
                    airport_names = [elem.get_text(strip=True) for elem in airport_name_elems]
                else:
                    airport_names = np.nan

                # Lấy danh sách khoảng cách
                # Ở đây ta tìm các thẻ chứa class có "qWPrE" – trong mẫu, khoảng cách nằm trong khối có class "biGQs _P pZUbB qWPrE hmDzD"
                distance_elems = airport_container.find_all("div", class_=lambda c: c and "qWPrE" in c)
                airport_distances = []
                if distance_elems:
                    for elem in distance_elems:
                        # Lấy toàn bộ text, sau đó dùng regex để trích xuất khoảng cách theo mẫu số + "mi"
                        text = elem.get_text(" ", strip=True)
                        m = re.search(r"(\d+(?:\.\d+)?)\s*mi", text)
                        if m:
                            airport_distances.append(m.group(1) + " mi drive")
                        else:
                            airport_distances.append("nan")
                else:
                    airport_distances = np.nan
            else:
                airport_names = np.nan
                airport_distances = np.nan

            # Nếu danh sách rỗng, gán lại là np.nan
            if isinstance(airport_names, list) and not airport_names:
                airport_names = np.nan
            if isinstance(airport_distances, list) and not airport_distances:
                airport_distances = np.nan

            # --- Lấy danh sách cho các nhóm: Hotels, Restaurants, Attractions ---
            group_results = {
                "Hotels": [],
                "Restaurants": [],
                "Attractions": []
            }
            hotel_count = restaurant_count = attraction_count = np.nan

            # Giả sử các nhóm được hiển thị trong container có class "DHAlN e H k _u"
            for container in soup.find_all("div", class_="DHAlN e H k _u"):
                # Lấy tiêu đề từ thẻ có class "CllfH"
                header = container.find("div", class_="CllfH")
                if not header:
                    continue
                header_text = header.get_text(strip=True)
                # Xác định nhóm dựa trên tiêu đề
                if "Hotels" in header_text:
                    group_category = "Hotels"
                elif "Restaurants" in header_text:
                    group_category = "Restaurants"
                elif "Attractions" in header_text:
                    group_category = "Attractions"
                else:
                    continue

                # Lấy số lượng mục (ví dụ: "228 Restaurants")
                count_match = re.search(r"(\d+)", header_text)
                if count_match:
                    count_val = int(count_match.group(1))
                    if group_category == "Hotels":
                        hotel_count = count_val
                    elif group_category == "Restaurants":
                        restaurant_count = count_val
                    elif group_category == "Attractions":
                        attraction_count = count_val

                # Duyệt qua từng mục trong container (mỗi mục thường nằm trong thẻ có class "Nhijf")
                for item in container.find_all("div", class_="Nhijf"):
                    # Thẻ đầu tiên chứa thông tin cơ bản
                    details_a = item.find("a", class_=lambda c: c and "BMQDV" in c)
                    if not details_a:
                        continue
                    detail_url = details_a.get("href", np.nan)
                    name_elem = details_a.find("div", class_=lambda c: c and "o W q" in c)
                    group_name = name_elem.get_text(strip=True) if name_elem else np.nan

                    # Lấy rating: thường tìm phần chứa cả "pZUbB" và "hmDzD"
                    rating_elem = item.find("div", class_=lambda c: c and "pZUbB" in c and "hmDzD" in c)
                    group_rating = rating_elem.get_text(strip=True) if rating_elem else np.nan

                    # Lấy số review (nếu có)
                    review_count = np.nan
                    review_parent = item.find("div", class_=lambda c: c and "nKWJn" in c)
                    if review_parent:
                        review_elem = review_parent.find("div", class_=lambda c: c and "pZUbB" in c and "hmDzD" in c)
                        if review_elem:
                            review_count = review_elem.get_text(strip=True).strip("()")

                    # --- Lấy thời gian đi bộ (walk_time) ---
                    walk_time = np.nan
                    # Tìm phần chứa walk_time trong container, ví dụ trong thẻ có class "xARtZ"
                    walk_div = item.find("div", class_="xARtZ")
                    if walk_div:
                        number_elem = walk_div.find("span", class_="number")
                        unit_elem = walk_div.find("span", class_="unit")
                        if number_elem and unit_elem:
                            walk_time = number_elem.get_text(strip=True) + " " + unit_elem.get_text(strip=True)

                    group_results[group_category].append({
                        "detail_url": detail_url,
                        "name": group_name,
                        "rating": group_rating,
                        "review_count": review_count,
                        "walk_time": walk_time
                    })

            # Nếu không tìm thấy mục nào, gán group_results thành list rỗng cho từng category
            for key in group_results:
                if not group_results[key]:
                    group_results[key] = []

            
            
            # Tạo ID
            hotel_id = f'HT{str(hotel_id_counter).zfill(6)}'
            hotel_id_counter += 1
            
            # Sau khi đã xử lý dữ liệu, bạn cần tạo dictionary để ghi vào CSV theo headers đã định nghĩa.
            csv_data = {
                "Hotel ID": hotel_id,
                "Country": country,
                "Tourist Attraction": place,
                "Name": name,
                "Address": address,
                "Price": price,
                "Phone Number": phone_number,
                "Website": website,
                "Ranking": ranking,
                "Description": description,
                "Room Features": ", ".join(room_features) if isinstance(room_features, list) and room_features else np.nan,
                "Room Types": ", ".join(room_types) if isinstance(room_types, list) and room_types else np.nan,
                "Hotel Class": hotel_class,
                "Languages Spoken": language_spoken,
                "Hotel Style": hotel_style,
                "Overall Rating Value": overall_rating_value,
                "Overall Rating Text": overall_rating_text,
                "Overall Review Count": overall_review_count,
                "Excellent Reviews": excellent_reviews,
                "Very Good Reviews": very_good_reviews,
                "Average Reviews": average_reviews,
                "Poor Reviews": poor_reviews,
                "Terrible Reviews": terrible_reviews,
                "Walker Score": walker_score,
                "Airport Names": ", ".join(str(x) for x in airport_names if not (isinstance(x, float) and np.isnan(x))) if isinstance(airport_names, list) and airport_names else np.nan,
                "Airport Distances": ", ".join(str(x) for x in airport_distances) if isinstance(airport_distances, list) and airport_distances else np.nan,
                "Near Hotels": hotel_count,
                "Near Restaurants": restaurant_count,
                "Near Attractions": attraction_count,
                "Hotels Group": group_results["Hotels"],
                "Restaurants Group": group_results["Restaurants"],
                "Attractions Group": group_results["Attractions"]
            }
            
            writer.writerow(csv_data)
            pbar.update(1)
